In [4]:
import torch
import torch.nn as nn
from torchvision import models
from torch.nn.functional import relu


import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [9]:
def dilation_block(in_channels,out_channels):
    block = Sequential(
    Conv2d(in_channels,out_channels,kernel_size=(3,3),stride=1,padding=1,dilation=1), #A dilation of 1 requires only padding = 1 with kernelsize 3
    BatchNorm2d(out_channels),
    ReLU(inplace=True),
    Conv2d(out_channels,out_channels,kernel_size=(3,3),stride=1,padding=1,dilation=1),
    BatchNorm2d(out_channels),
    ReLU(inplace=True),
    Conv2d(out_channels,out_channels,kernel_size=(3,3),stride=1,padding=2,dilation=2),
    BatchNorm2d(out_channels),
    ReLU(inplace=True),
    Conv2d(out_channels,out_channels,kernel_size=(3,3),stride=1,padding=2,dilation=2),
    BatchNorm2d(out_channels),
    ReLU(inplace=True)
    )
    return block

from torch.nn import Conv2d,Sequential, ReLU, Linear, MaxPool2d, BatchNorm2d, ConvTranspose2d
class L_U_Net(nn.Module):
    def __init__(self,num_classes,num_filters) -> None:
        super().__init__()
        self.initial_layer = dilation_block(3,num_filters)
        self.dil_block = dilation_block(num_filters,num_filters)
        self.pooling = MaxPool2d(kernel_size=(2,2),stride=(2,2))

        self.bottleneck = nn.Sequential(
            nn.Conv2d(num_filters, num_filters, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(inplace=True),

            nn.Conv2d(num_filters, num_filters, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(inplace=True),
        )

        self.expand_tconv = ConvTranspose2d(num_filters,num_filters,kernel_size=(2,2),stride=2)

        self.expand_conv = Sequential(
            nn.Conv2d(num_filters*2, num_filters, kernel_size=3, stride=1, padding=1), # numfilters*2 due to skip connections :)
            nn.BatchNorm2d(num_filters),
            nn.ReLU(inplace=True),

        )
        self.final_layer = Conv2d(num_filters,num_classes, kernel_size=1)
    def forward(self,x):
        dil1 = self.initial_layer(x)
        pool1 = self.pooling(dil1)

        dil2 = self.dil_block(pool1)
        pool2 = self.pooling(dil2)

        dil3 = self.dil_block(pool2)
        pool3 = self.pooling(dil3)

        dil4 = self.dil_block(pool3)
        pool4 = self.pooling(dil4)

        bottle_neck = self.bottleneck(pool4)

        tconv1 = self.expand_tconv(bottle_neck)
        skip1 = torch.cat((tconv1,dil4),dim=1)
        conv1= self.expand_conv(skip1)

        tconv2= self.expand_tconv(conv1)
        skip2 = torch.cat((tconv2,dil3),dim=1)
        conv2 = self.expand_conv(skip2)

        tconv3 = self.expand_tconv(conv2)
        skip3 = torch.cat((tconv3,dil2),dim=1)
        conv3 = self.expand_conv(skip3)

        tconv4 = self.expand_tconv(conv3)
        skip4 = torch.cat((tconv4,dil1),dim=1)
        conv4 = self.expand_conv(skip4)

        return self.final_layer(conv4)


In [11]:
# Example usage
input_tensor = torch.randn(6, 3, 224, 224)  # (batch_size, channels, height, width)
model = L_U_Net(num_classes=10, num_filters=16)
output_tensor = model(input_tensor)

print(output_tensor.size())  # Output dimensions should match the input spatial dimensions but with num_classes channels

torch.Size([6, 10, 256, 256])


In [13]:
import torch
import torch.nn as nn

def conv_block(in_channels, out_channels):
    block = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )
    return block
def up_conv(in_channels, out_channels):
    return  nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

class UNet(nn.Module):
    def __init__(self, num_classes):
        super(UNet, self).__init__()
        self.encoder1 = conv_block(3, 64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.encoder2 = conv_block(64, 128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.encoder3 = conv_block(128, 256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.encoder4 = conv_block(256, 512)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = conv_block(512, 1024)

        self.upconv1 = up_conv(1024, 512)
        self.decoder1 = conv_block(1024, 512)

        self.upconv2 = up_conv(512, 256)
        self.decoder2 = conv_block(512, 256)

        self.upconv3 = up_conv(256, 128)
        self.decoder3 = conv_block(256, 128)

        self.upconv4 = up_conv(128, 64)
        self.decoder4 = conv_block(128, 64)

        self.final_layer = nn.Conv2d(64, num_classes, kernel_size=1)

    def forward(self, x):
        # Contracting Path
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        # Bottleneck
        bottleneck = self.bottleneck(self.pool4(enc4))

        # Expanding Path




        dec1 = self.upconv1(bottleneck)
        skip1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(skip1)

        dec2 = self.upconv2(dec3)
        skip2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(skip2)



        dec3 = self.upconv3(dec4)
        skip3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(skip3)

        dec4 = self.upconv4(dec3)
        skip4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(skip4)

        return self.final_layer(dec1)

input_tensor = torch.randn(1, 3, 224, 224)  # (batch_size, channels, height, width)
model = UNet(num_classes=10)
output_tensor = model(input_tensor)

print(output_tensor.size())  # Output dimensions should be [1, num_classes, 224, 224]


torch.Size([1, 10, 224, 224])
